In [1]:
!nvidia-smi

Sun Sep 26 18:55:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 13.0 MB/s 
     |████████████████████████████████| 3.3 MB 42.9 MB/s 
     |████████████████████████████████| 52 kB 1.4 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 636 kB 31.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/My Drive/RR-project/NL2GraphQuery-workplace/code/NLtoGQ')

In [5]:
from transformers import RobertaTokenizer, RobertaModel
import sys,os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import numpy as np
import pandas as pd

import random
import math
import time

from multiHop_QA.configures import Config_path,Config_output_path,Hyparams_transformers

In [6]:
config = Config_path()
config_output = Config_output_path()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

# BERT embedding transformer

In [ ]:
from multiHop_QA.train_RoBERTa_emb import QuesVocabs,RelVocabs

train_combination = config.train_combination_path
test_combination = config.test_combination_path

m_q_vocabs = QuesVocabs(train_combination,test_combination)
q2id, id2q = m_q_vocabs.get_ques_vocabs()

rel_vocabs = RelVocabs(train_combination,test_combination)
rel2id, id2rel = rel_vocabs.get_rl_vocabs()


def query_transformer_emb(model, sentence, rel2id, id2rel, max_len=256):
    src_indexes = tokenizer(sentence)['input_ids']
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = model.make_src_mask(src_tensor)
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)
        # print(enc_src)
    trg_indexes = [rel2id["[CLS]"]]
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        # print('trg_mask',trg_mask)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
            # print(output,attention)
        pred_token = output.argmax(2)[:, -1].item()
        # print(output.argmax(2),output.shape)
        trg_indexes.append(pred_token)
        if pred_token == rel2id["[SEP]"]:
            break
    trg_tokens = [id2rel[i] for i in trg_indexes]
    return trg_tokens[1:-1]

In [ ]:
model_emb = torch.load(os.path.join(config_output.transformers_path, '0923_BERT_emb_transformers_qspace_best-model-1e5lr.pt'))

In [ ]:
query_transformer_emb(model_emb, 'what is the country of origin of the show [UNK]', rel2id, id2rel, max_len=256)

['people/person/nationality']

In [ ]:
query_transformer_emb(model_emb, 'who is the manufacturer of [UNK]', rel2id, id2rel, max_len=256)

tensor([[[ 0.5382,  0.2331, -0.1027,  ..., -0.9405, -1.4228, -0.0429],
         [ 0.5434,  0.2343, -0.1104,  ..., -0.9408, -1.4191, -0.0390],
         [ 0.5401,  0.2327, -0.1068,  ..., -0.9386, -1.4208, -0.0368],
         ...,
         [ 0.5435,  0.2328, -0.1014,  ..., -0.9391, -1.4197, -0.0409],
         [ 0.5405,  0.2327, -0.1074,  ..., -0.9405, -1.4219, -0.0408],
         [ 0.5413,  0.2365, -0.1048,  ..., -0.9405, -1.4201, -0.0348]]],
       device='cuda:0')
tensor([[22]], device='cuda:0') torch.Size([1, 1, 1781])
tensor([[22,  2]], device='cuda:0') torch.Size([1, 2, 1781])


['[CLS]', 'people/person/nationality', '[SEP]']

# vanilla  transformer

In [7]:
from multiHop_QA.train import QuesVocabs,RelVocabs

train_combination = config.train_combination_path
test_combination = config.test_combination_path

m_q_vocabs = QuesVocabs(train_combination,test_combination)
q2id, id2q = m_q_vocabs.get_ques_vocabs()

rel_vocabs = RelVocabs(train_combination,test_combination)
rel2id, id2rel = rel_vocabs.get_rl_vocabs()

def query_transformer(sentence, q2id, rel2id, id2rel, model, device, max_len=256):
    model.eval()
    sentence = "[CLS]" + sentence + "[SEP]"
    if isinstance(sentence, str):
        tokens = tokenizer.tokenize(sentence)
    else:
        tokens = [token.lower() for token in sentence]
    # tokens = sentence.split()
    # tokens = [token.lower() for token in tokens]
    src_indexes = [q2id[token] for token in tokens]
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = model.make_src_mask(src_tensor)
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)
        # print(enc_src)
    trg_indexes = [rel2id["[CLS]"]]
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
            # print(output)
        pred_token = output.argmax(2)[:, -1].item()
        trg_indexes.append(pred_token)
        if pred_token == rel2id["[SEP]"]:
            break
    trg_tokens = [id2rel[i] for i in trg_indexes]
    return trg_tokens[1:-1], attention

In [8]:
# model = torch.load(os.path.join(config_output.transformers_path, '0924_transformers_qsapce_best-model-1e4lr.pt'))
model = torch.load(os.path.join(config_output.transformers_path, '0924_transformers_q_sep_best-model-1e4lr.pt'))


In [9]:
# # print predicted result
sentence = "what is the country of origin of the show [UNK]"
# print(sentence.split())
prediction, attention = query_transformer(sentence, q2id, rel2id, id2rel, model, device)
print(f'predicted trg = {prediction}')

predicted trg = ['tv/tv_program/country_of_origin']


In [10]:
# # print predicted result
sentence = "who is the manufacturer of [UNK]"
# print(sentence.split())
prediction, attention = query_transformer(sentence, q2id, rel2id, id2rel, model, device)
print(f'predicted trg = {prediction}')

predicted trg = ['aviation/aircraft_model/manufacturer']


# vanilla transformer metrics

In [11]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
test_combination = pd.read_csv(config.test_combination_path)

In [12]:
n = len(test_combination)//2
test_x = test_combination['q_space'][n:-1].values
test_y = test_combination['relationship'][n:-1].values

In [13]:
test_y_list = []
pre_y_list = []
for i in range(len(test_x)):
  try:
    pre_y, attention = query_transformer(test_x[i], q2id, rel2id, id2rel, model, device)
    test_y_list.append(test_y[i])
    pre_y_list.append(pre_y)
  except:
    pass

In [14]:
f1_macro = f1_score(test_y_list, pre_y_list, average='macro')
f1_micro = f1_score(test_y_list, pre_y_list, average='micro')
f1_weighted = f1_score(test_y_list, pre_y_list, average='weighted')
test_accuracy = accuracy_score(test_y_list, pre_y_list)
print('f1_macro = {f1_macro:.2f}, f1_micro = {f1_micro:.2f}, f1_weighted = {f1_weighted:.2f}, test_accuracy = {test_accuracy:.2f}'.format(f1_macro=f1_macro,f1_micro=f1_micro,f1_weighted=f1_weighted,test_accuracy=test_accuracy))

f1_macro = 0.31, f1_micro = 0.69, f1_weighted = 0.67, test_accuracy = 0.69
